# PYTS/BOSSVS classification algorithm test and measured accuracies
# Tested on transformed data

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import math
import sys 
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)       

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [6]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
from sklearn.metrics import f1_score
iteration=100
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=2

ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):



    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]



if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/5):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

# gg = np.unique(Q_ON)
# Q_ON2=[]
# if len(gg)>0:
#     for i in range(len(gg)):
#         aa=gg[i]
#         bb=test_index[aa]
#         name=file_names_ALL[bb]
#         count = Q_ON.count(bb)
#         Q_ON2.append([name,count])   
# print(Q_ON2)

(1089, 2, 37)
Accuracy for ON class:  47.3554704150173 %
Accuracy for OFF class:  77.75177430475993 %
False Positive rate:  52.6445295849827 %
False Negative rate:  22.24822569524007 %
F1 score:  0.7898561778067062
[]
